In [ ]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import numpy as np 
from matplotlib import pyplot as plt
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import random_split
from torchvision import models,datasets
import os
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from PIL import Image
from torch.autograd import Variable

In [ ]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    print('using device: cuda')
else:
    print('using device: cpu')

device = torch.device("cuda:0" if USE_GPU else "cpu")

In [ ]:
#Hyperparamters
BATCH_SIZE = 64

In [ ]:
train_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        #transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        #transforms.ColorJitter(),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

test_transform = transforms.Compose([
        transforms.Resize((256,256)),
        #transforms.CenterCrop(256),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])

In [ ]:
class Flare_Classifier(Dataset):
    
    def __init__(self,directory,filelist,c,transform = None):
        self.filelist = filelist
        self.directory = directory
        self.c = c
        self.transform = transform
    
    def __len__(self):
        return len(self.filelist)
    
    def __getitem__(self,idx):
        img =  Image.open(os.path.join(self.directory, self.filelist[idx])).convert("RGB")
        img = self.transform(img)
        
        if 'Flare_img' == self.c:
            self.label = 0
        elif 'Without_Flare_' == self.c:
            self.label = 1
        
        img = img.numpy()
        return img, self.label

In [ ]:
parent_dir = '../input/flaredataset/Flare'
classes = ['Flare_img','Without_Flare_']
working_ds = None
for c in classes:
    directory = os.path.join(parent_dir,c)
    print(directory)
    file_list = os.listdir(directory)
    new = Flare_Classifier(directory,file_list,c,train_transform)
    
    if working_ds == None:
        working_ds = new
    else:
        working_ds = ConcatDataset([working_ds,new])
        
print(len(working_ds))

In [ ]:
def train_val_split(working_ds):
    val_split = 0.05
    l = float(len(working_ds))
    m = int(val_split*l)
    train_ds,val_ds = random_split(working_ds,[int(l)-m,m])
    return train_ds,val_ds

train_ds,val_ds = train_val_split(working_ds)

train_loader_ = torch.utils.data.DataLoader(dataset=train_ds,
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True)

val_loader_ = torch.utils.data.DataLoader(dataset=val_ds,
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy
samples, labels = iter(train_loader_).next()
plt.figure(figsize=(16,24))
grid_imgs = torchvision.utils.make_grid(samples[:24], normalize  = True)
np_grid_imgs = grid_imgs.numpy()
# in tensor, image is (batch, width, height), so you have to transpose it to (width, height, batch) in numpy to show it.
plt.imshow(numpy.transpose(np_grid_imgs, (1,2,0)))
print(labels)


In [ ]:
fe = models.squeezenet1_0(pretrained=True)
for param in fe.parameters():
	param.requires_grad = True
class ResNet18(nn.Module):
  def __init__(self):
    super(ResNet18, self).__init__()
    self.features = torch.nn.Sequential(*list(fe.children()))
    self.conv1 = nn.Conv2d(1000,3, 3,1,1)
    self.pool =  nn.AdaptiveAvgPool2d(output_size=(1,1))
    self.drop1 = nn.Dropout(0.3)
    self.fc1 = nn.Linear(3,128)
    self.drop2 = nn.Dropout(0.2)
    self.fc2 = nn.Linear(128,1)
    #We did not add a softmax layer here because the CrossEntropy Loss function contains a softmax, so if you want 
    #to test output, you will have to add a softmax block in addition to the model block
    
  def forward(self,x):
    x = self.features(x)
    x = self.conv1(x)
    x = self.pool(x)
    x = x.view(x.shape[0],3)
    
    x = self.drop1(x)
    x = F.relu(self.fc1(x))
    x = self.drop2(x)
    x = F.sigmoid(self.fc2(x))    
    return x

criterion_cl = nn.MSELoss()    

resnet_model = ResNet18()
resnet_model = resnet_model.to(device)

In [ ]:
c1 = resnet_model.features
c2 = resnet_model.conv1

In [ ]:
!pip install torchsummary
from torchsummary import summary
summary(resnet_model,(3,512,512))

In [ ]:
f,w = next(iter(train_loader_))
f = f.to(device).float()
w = w.to(device).float()
i = c1(f)
j = c2(i).detach()

plt.figure(figsize=(8,8))
img_grid = torchvision.utils.make_grid(j.cpu(),normalize = True)
plt.imshow(np.transpose(img_grid,(1,2,0)))
plt.show()
print(f.dtype)
print(w.dtype)

In [ ]:
def model_eval(loader, model):
        data = next(iter(loader))
        i,trues = data
        i=i.to(device)
        trues=trues.to(device)
        preds = model(i).cpu().detach()

        preds = [1 if preds[i] >= 0.5 else 0 for i in range(len(preds))]

        acc = [1 if preds[i] == trues[i] else 0 for i in range(len(preds))]

        acc = np.sum(acc) / len(preds)
    
        return (acc * 100)

In [ ]:
print(model_eval(val_loader_,resnet_model))


In [ ]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
history =[]
train_acc =[]
val_acc = []
def train_classifier(model, criterion, optimizer,sch,num_epochs,train_loader,val_loader):
  
  for epoch in range(num_epochs): 
      print("Epoch:",epoch+1)
      running_loss = 0.0
      loop = tqdm(train_loader, leave = True)
      for i, data in enumerate(loop):
          inputs,labels = data
          inputs = inputs.to(device)
          labels = labels.to(device).to(float)

          optimizer.zero_grad()
    
          output = model(inputs)
          output = output.to(device).to(float).squeeze()
          loss = criterion(output, labels)
          
          loss.backward()
          optimizer.step()
          loop.set_postfix(loss = loss.item())
          
          running_loss += loss.item()

          if i % 100 == 99:    
            history.append(running_loss)
            train_acc.append(model_eval(train_loader, model))
            val_acc.append(model_eval(val_loader, model))
            running_loss = 0.0

      sch.step()  
      print('Epoch:', epoch+1,'LR:', sch.get_last_lr())
      print("EPOCH OVER")
    #  train_acc = model_eval(train_loader,model)
      with torch.no_grad():
        test_acc = model_eval(val_loader,model)
      print("Testing Accuracy",test_acc)
  return model

In [ ]:
from torch.optim import lr_scheduler
criterion = nn.BCELoss() 
num_epochs = 40
optimizer = optim.SGD(resnet_model.parameters(), lr=1e-4, momentum=0.95)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
train_classifier(resnet_model, criterion, optimizer, scheduler, num_epochs , train_loader_, val_loader_)

In [ ]:
model_save_name = 'Flare_Classifier_25_epochs_with_metrics.pt'
path = F".//{model_save_name}" 
torch.save(resnet_model.state_dict(), path)

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Loss Vs Iterations")
plt.plot(history, color = 'blue', label = 'Training Loss')
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Accuracy Vs Iterations")
plt.plot(train_acc, color = 'blue', label = 'Training Accuracy')
plt.plot(val_acc, color = 'yellow', label = 'Validation Accuracy')
plt.xlabel("Iterations")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
import pickle
with open("tain_acc.txt", "wb") as fp:   #Pickling
   pickle.dump(train_acc, fp)

with open("val_acc.txt", "wb") as fp:   # Unpickling
   pickle.dump(val_acc, fp)

with open("history.txt", "wb") as fp:   # Unpickling
   pickle.dump(history, fp)

In [ ]:
from sklearn.metrics import f1_score
images, labels = next(iter(val_loader_))
images = images.to(device)
outputs = resnet_model(images)
print(images.dtype)
print(labels.dtype)
for i in range(len(labels)):
    outputs[i] = torch.round(outputs[i])
outputs = outputs.cpu().detach().numpy()
count = 0
score = 0
for i,data in enumerate(val_loader_):
    for i in range(len(labels)):
        score += f1_score(labels, outputs)
        count+=1
print(score/count)

In [ ]:
f,w = next(iter(train_loader_))
f = f.to(device).float()
w = w.to(device).float()
i = c1(f)
j = c2(i).detach()

plt.figure(figsize=(16,8))
img_grid = torchvision.utils.make_grid(f[:16].cpu(),normalize = True)
plt.imshow(np.transpose(img_grid,(1,2,0)))
plt.show()

plt.figure(figsize=(16,8))
img_grid = torchvision.utils.make_grid(j[:16].cpu(),normalize = True)
plt.imshow(np.transpose(img_grid,(1,2,0)))
plt.show()

out = resnet_model(f)
print(out)